In [1]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import os

selected_line =['8700-10-1',
                '7545-10-1',
                '7545-10-0',   #Less two links
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',   #Less two links
                '2290-10-1',
                '2290-10-0',
                '477P-10-0',        
                '3301-10-0',   #Less two links
                '574J-10-1',   #Less two links
                '574J-10-0',   #Less two links
                '477P-10-1',   #Less two links
                '351F-10-1',
                '351F-10-0'] 

selected_line = ['6450-10-0']

### Size of the steps to group
frequencies = ['20min', '30min', '1H', '3H', '1d', '7d', '1m']
frequencies = ['30min']

In [2]:
for line in selected_line:  
    filename = str(line) + '_temp.csv.gz'
    df = pd.read_csv(filename, compression='gzip', sep=',')
    df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
    df.index = df['exact_time']
    
    df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())
    
    start_date = pd.to_datetime('2017-1-1', format = '%Y-%m-%d')
    end_date = pd.to_datetime('2017-9-25', format = '%Y-%m-%d')
    df = df.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]
 
    
    frequency = '60min'            
    rolling_win = 3
    df = df.drop(df[df['link'] == max(df['link'])].index)
    if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
            (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
            df = df.drop(df[df['link'] == max(df['link'])].index)

    X_Temp = df.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).mean()['time_link'].unstack()    
    X_Temp = X_Temp.transform(lambda x: x.fillna(method='ffill')).dropna()
    X_Temp.describe()

With one lag used for prediction

In [35]:
import numpy as np
from keras.layers import Input
from keras.layers import Dense
from keras.models import Model

input_dim = 27

input_data = Input(shape=(input_dim,))
layer1 = Dense(20, activation='relu')(input_data)
layer2 = Dense(20, activation='relu')(layer1)
layer3 = Dense(input_dim, activation='sigmoid')(layer2)

# this model maps an input to its reconstruction
nn = Model(input_data, layer3)

# Creates the train and test sets 
test_samples = 500
train_X = X_Temp.values[:-test_samples,:-2][:-1]
train_Y = X_Temp.shift(-1).values[:-test_samples,:-2][:-1]
test_X = X_Temp.values[-test_samples:,:-2][:-1]
test_Y = X_Temp.shift(-1).values[-test_samples:,:-2][:-1]

#Normalize the inputs
trmin = train_X.min(); trmax = train_X.max();
temin = test_X.min(); temax = test_X.max();
train_X = (train_X-trmin)/(trmax-trmin)
test_X  = (test_X-temin)/(temax-temin)
train_Y = (train_Y-trmin)/(trmax-trmin)
test_Y  = (test_Y-temin)/(temax-temin)

#print(test_X)
#print(test_Y)


With 3 lags used as predictions

In [60]:
import numpy as np
from keras.layers import Input
from keras.layers import Dense
from keras.models import Model

input_dim = 27*3
output_dim = 27

input_data = Input(shape=(input_dim,))
layer1 = Dense(60, activation='relu')(input_data)
layer2 = Dense(40, activation='relu')(layer1)
layer3 = Dense(output_dim, activation='sigmoid')(layer2)

# this model maps an input to its reconstruction
nn = Model(input_data, layer3)

# Creates the train and test sets 
test_samples = 500
train_X = np.concatenate([X_Temp.values[:-test_samples,:-2][:-3], X_Temp.shift(-1).values[:-test_samples,:-2][:-3], X_Temp.shift(-2).values[:-test_samples,:-2][:-3]], axis=1)
train_Y = X_Temp.shift(-3).values[:-test_samples,:-2][:-3]
test_X  = np.concatenate([X_Temp.values[-test_samples:,:-2][:-3], X_Temp.shift(-1).values[-test_samples:,:-2][:-3], X_Temp.shift(-2).values[-test_samples:,:-2][:-3]], axis=1)
test_Y  = X_Temp.shift(-3).values[-test_samples:,:-2][:-3]

#Normalize the inputs
trmin = train_X.min(); trmax = train_X.max();
temin = test_X.min(); temax = test_X.max();
train_X = (train_X-trmin)/(trmax-trmin)
test_X  = (test_X-temin)/(temax-temin)
train_Y = (train_Y-trmin)/(trmax-trmin)
test_Y  = (test_Y-temin)/(temax-temin)

#print(test_X)
#print(test_Y)

print (train_X.shape)
print (train_Y.shape)
print (test_X.shape)
print (test_Y.shape)

(2473, 81)
(2473, 27)
(497, 81)
(497, 27)


In [64]:
nn.compile(optimizer='adam', loss='binary_crossentropy') #mean_squared_error

nn.fit(train_X, train_Y,
                epochs=200,
                batch_size=128,
                shuffle=True,
                validation_data=(test_X, test_Y))

Train on 2473 samples, validate on 497 samples
Epoch 1/200
2473/2473 [==============================] - 1s 235us/step - loss: 0.6504 - val_loss: 0.6359
Epoch 2/200
2473/2473 [==============================] - 0s 19us/step - loss: 0.6502 - val_loss: 0.6353
Epoch 3/200
2473/2473 [==============================] - 0s 23us/step - loss: 0.6501 - val_loss: 0.6346
Epoch 4/200
2473/2473 [==============================] - 0s 27us/step - loss: 0.6501 - val_loss: 0.6352
Epoch 5/200
2473/2473 [==============================] - 0s 25us/step - loss: 0.6501 - val_loss: 0.6362
Epoch 6/200
2473/2473 [==============================] - 0s 22us/step - loss: 0.6501 - val_loss: 0.6394
Epoch 7/200
2473/2473 [==============================] - 0s 24us/step - loss: 0.6502 - val_loss: 0.6342
Epoch 8/200
2473/2473 [==============================] - 0s 21us/step - loss: 0.6501 - val_loss: 0.6320
Epoch 9/200
2473/2473 [==============================] - 0s 22us/step - loss: 0.6502 - val_loss: 0.6372
Epoch 10/200
247

2473/2473 [==============================] - 0s 23us/step - loss: 0.6499 - val_loss: 0.6345
Epoch 156/200
2473/2473 [==============================] - 0s 21us/step - loss: 0.6499 - val_loss: 0.6353
Epoch 157/200
2473/2473 [==============================] - 0s 21us/step - loss: 0.6499 - val_loss: 0.6374
Epoch 158/200
2473/2473 [==============================] - 0s 22us/step - loss: 0.6499 - val_loss: 0.6349
Epoch 159/200
2473/2473 [==============================] - 0s 20us/step - loss: 0.6499 - val_loss: 0.6327
Epoch 160/200
2473/2473 [==============================] - 0s 22us/step - loss: 0.6499 - val_loss: 0.6369
Epoch 161/200
2473/2473 [==============================] - 0s 24us/step - loss: 0.6499 - val_loss: 0.6379
Epoch 162/200
2473/2473 [==============================] - 0s 22us/step - loss: 0.6499 - val_loss: 0.6355
Epoch 163/200
2473/2473 [==============================] - 0s 23us/step - loss: 0.6499 - val_loss: 0.6360
Epoch 164/200
2473/2473 [==============================] - 0

In [65]:
predicted_Y = nn.predict(test_X)

predicted_Y = predicted_Y *(temax-temin)+temin
test_Y_orig = test_Y*(temax-temin)+temin

#state_i = 0
#print(test_orig[state_i])
#print(decoded_states[state_i])
#print((decoded_states[state_i]-test_orig[state_i])/test_orig[state_i])
#state_i = 1
#print((decoded_states[state_i]-test_orig[state_i])/test_orig[state_i])
#state_i = 2
#print((decoded_states[state_i]-test_orig[state_i])/test_orig[state_i])


errors = (predicted_Y-test_Y_orig)/test_Y_orig
#errors_pd = pd.DataFrame(errors[:,:-2]) # two last columns have large errors
errors_pd = pd.DataFrame(errors)
print(np.mean((errors_pd).mean()))
errors_pd.describe()

#pd.DataFrame(test_orig).describe()

0.13970977990304304


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
count,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,...,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000,497.000000
mean,0.182540,0.257947,0.320552,0.179098,0.142605,0.091544,0.107095,0.240441,0.209236,0.240531,...,0.126007,0.084449,0.119571,0.113133,0.035102,0.032206,0.065886,0.096490,0.069884,0.174531
std,0.275059,0.296630,0.298311,0.256946,0.239671,0.219320,0.225338,0.261756,0.249563,0.264407,...,0.228527,0.196474,0.205552,0.211524,0.191065,0.244674,0.271120,0.298961,0.252942,0.172129
min,-0.281662,-0.345910,-0.339419,-0.267909,-0.325877,-0.210831,-0.225793,-0.310022,-0.336349,-0.220613,...,-0.575518,-0.365920,-0.377531,-0.341576,-0.404300,-0.442475,-0.492857,-0.468869,-0.393229,-0.419008
25%,0.024335,0.072967,0.136543,0.019629,-0.003994,-0.046704,-0.033349,0.070677,0.043525,0.069099,...,-0.006359,-0.028591,-0.015097,-0.030935,-0.075925,-0.111386,-0.104168,-0.118123,-0.089073,0.073835
50%,0.113433,0.181833,0.242254,0.098587,0.064306,0.015180,0.031995,0.156133,0.132325,0.159409,...,0.082747,0.057239,0.092440,0.078661,-0.003290,-0.005161,0.017510,0.027797,0.018630,0.163404
75%,0.241599,0.328317,0.391086,0.245689,0.209382,0.146556,0.166435,0.305755,0.271777,0.309022,...,0.202356,0.150687,0.216179,0.202878,0.119097,0.135421,0.182658,0.237216,0.151416,0.253019
max,1.939179,1.917220,1.898473,1.489320,1.245764,1.011221,1.163970,1.461200,1.361247,1.455762,...,1.821447,1.557963,1.141022,1.051108,1.306108,2.471442,2.349229,1.371522,1.356971,1.098934
